# Import libraries
Primerament, importem les llibreries. I carregar el learner de les fruites ja entrenat.

In [1]:
from fastai.vision.all import *

In [2]:
learner = load_learner('./learns/fruits.pkl')

In [3]:
dls = ImageDataLoaders.from_folder(
    "./dataset/train",
    valid_pct=0.2,
    seed=42,
    item_tfms=Resize(224)
)

# Prediure Imatges
Podem prediure imatges per comprovar el funcionament de learner.

In [4]:
image = PILImage.create("./dataset/train/Apple/Apple (5).png")
image.show()

In [5]:
learner.predict(image)

# Grad-CAM
Trobar la forma de fer funcionar la Grad-CAM

In [6]:
x, = first(dls.test_dl([image]))

In [7]:
class Hook:
    def hook_func(self, m, i, o):
        self.stored = o.detach().clone()

In [8]:
hook_output = Hook()
hook = learner.model[0].register_forward_hook(hook_output.hook_func)

In [9]:
with torch.no_grad(): 
    output = learner.model.eval()(x)

In [10]:
act = hook_output.stored[0]
F.softmax(output, dim=-1)

In [11]:
x.shape

In [12]:
cam_map = torch.einsum('ck,kij->cij', learner.model[1][-1].weight, act)
cam_map.shape

In [13]:
x_dec = TensorImage(dls.train.decode((x,))[0][0])
_,ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map[1].detach().cpu(), alpha=0.6, extent=(0,224,224,0),
              interpolation='bilinear', cmap='jet');